In [1]:
import numpy as np
import faiss
import time
import pandas as pd
from os import path, makedirs
import sys

sys.path.append('../')
from utils import get_duplicate_dist

## Configuration Variables

In [2]:
root = '../../../inference_array/resnet50/'
model = 'mrl/' # mrl/, rr/

dataset = '1K' # 1K, 4K, V2
index_type = 'ivfpq' # ivfpq, ivfsq, opq, kmeans, pq
use_svd = False
    
use_gpu = 0 # GPU inference for exact search. Disable for CPU search
if use_gpu and faiss.get_num_gpus() == 0:
    raise Exception("GPU search is enabled but no GPU was found.")

## FAISS Index Building and NN Search

In [3]:
if model == "mrl/":
    config = 'mrl1_e0_ff2048'
elif model == 'rr/':
    config = 'mrl0_e0_ff'
else: 
    raise Exception("Unsupported pretrained model.")

db_npy = dataset + '_train_' + config + '-X.npy'
query_npy = dataset + '_val_' + config + '-X.npy'

# ImageNetv2 is only a test set; set database to ImageNet-1K
if dataset == 'V2':
    db_npy = '1K_train_' + config + '-X.npy'

print("Loading database: ", db_npy)
print("Loading queries:  ", query_npy)

Loading database:  1K_train_mrl1_e0_ff2048-X.npy
Loading queries:   1K_val_mrl1_e0_ff2048-X.npy


## SVD Ablation

In [4]:
def dim_reduce_pca(full_database, low_dim):
    subsampled_db = np.ascontiguousarray(full_database[0::10], dtype=np.float32)
    
    mat = faiss.PCAMatrix (subsampled_db.shape[1], low_dim)
    mat.train(subsampled_db)
    assert mat.is_trained
    
    return mat

def get_svd_data(svd_low_dim):
    print("Using PCA for low-dim projection with d= ", svd_low_dim)

    database = np.load(root+db_npy)
    queryset = np.load(root+query_npy)

    print("Original Database: ", database.shape)
    svd_mat = dim_reduce_pca(database, svd_low_dim)
    database_svd_lowdim = svd_mat.apply(database)
    print("Low-d Database: ", database_svd_lowdim.shape)
    query_svd_lowdim = svd_mat.apply(queryset)

    faiss.normalize_L2(database_svd_lowdim)
    faiss.normalize_L2(query_svd_lowdim)
    
    return database_svd_lowdim, query_svd_lowdim

if use_svd:
    svd_low_dim = 1024
    db_npy_svd = dataset+'_train_'+config+"_svd"+str(svd_low_dim)+"-X.npy"
    query_npy_svd = dataset+'_val_'+config+"_svd"+str(svd_low_dim)+"-X.npy"
    database_svd_lowdim, query_svd_lowdim = get_svd_data(svd_low_dim)
    
    if not path.exists(root+db_npy_svd):
        np.save(root+db_npy_svd, database_svd_lowdim)
    if not path.exists(root+query_npy_svd):
        np.save(root+query_npy_svd, query_svd_lowdim)

## Full Precision Embedding Loading (MRL)

In [5]:
if model == 'mrl/':
    if use_svd:
        xb = np.load(root+db_npy_svd)
        assert np.count_nonzero(np.isnan(xb)) == 0
        xq = np.load(root+query_npy_svd)
    else:
        xb = np.load(root+db_npy)
        assert np.count_nonzero(np.isnan(xb)) == 0
        xq = np.load(root+query_npy)

## Database Indexing and Search

In [6]:
k = 2048 # nearest neighbors shortlist length, default = 2048 = max value supported by FAISS
#for any pq index, iterator is m. For kmeans, iterator is number of centroids
iterator = [8, 16, 32, 64] # M for PQ
Ds = [8, 16, 32, 64, 128, 256, 512, 1024, 2048] # embedding dimensionality

# IVF index specific params
nprobes = [1] # number of search probes (cells to search)
nbits = 8 # nbits used to represent centroid id; total possible is k* = 2**nbits
nlist = 1024  # number of cells (must be >= k*)

In [7]:
# Return index file name of given dimensionality based on index type
def get_index_file(dim):
    if index_type in ['ivfpq', 'opq', 'pq']:
        size = 'm'+str(dim)+"_d"+str(D)
    elif index_type == 'kmeans':
        size = str(dim)+'ncentroid_'+str(D)+'d'
    elif index_type == 'kmeans-pca':
        size = str(dim)+'ncentroid_'+str(svd_low_dim)+'pcadim'
    else:
        raise Exception("Unsupported Index!")

    index_file = root+'index_files/'+model+index_type+"/"+dataset+'_'+index_type+'_'+size
    if index_type in ['ivfpq', 'ivfsq']:
        index_file += "_nbits"+str(nbits)+'_nlist'+str(nlist)+'.index'
    
    return index_file

In [8]:
# Iterate over embedding dimensionality of database and queries
for D in Ds:
    # Iterate over subquantizers (PQ) or num_centroids (kmeans)
    for dim in iterator:
        if (index_type in ['pq', 'ivfpq', 'opq']):
            m = dim # number of sub-vectors to divide d into
            if D % m != 0 or m > D:
                continue
        
        if not path.isdir(root+'index_files/'+model+'/'):
            makedirs(root+'index_files/'+model+'/')
        
        index_file = get_index_file(dim)           
            
        if model != 'mrl/':
            db_npy = dataset + '_train_' + config +str(D)+ '-X.npy'
            print("Loading database: ", db_npy)
            xb = np.load(root+db_npy)
            
        database = np.ascontiguousarray(xb[:,:D], dtype=np.float32)
        faiss.normalize_L2(database)
        print("Indexing database ", database.shape)
        print("Generating index file: " + index_file.split("/")[-1])

        # Load or build index
        if path.exists(index_file):
            print("Loading index file: " + index_file)
            cpu_index = faiss.read_index(index_file)

        else:
            if (index_type == 'pq'):
                cpu_index = faiss.IndexPQ(D, dim, nbits)
                cpu_index.train(database)
                            
            elif index_type in ['ivfpq', 'opq']:
                quantizer = faiss.IndexFlatL2(D) # L2 quantizer to assign vectors to Voronoi cells
                cpu_index = faiss.IndexIVFPQ(quantizer, D, nlist, m, nbits)

                # Learn a transformation of the embedding space with Optimized PQ
                if index_type == 'opq':
                    opq_matrix = faiss.OPQMatrix(D, dim)
                    cpu_index = faiss.IndexPreTransform (opq_matrix, cpu_index)

                print("Training %s Index:  d=%d, m=%d, nbits=%d, nlist=%d" %(index_type,D,m,nbits,nlist))
                ivfpq_start = time.time()
                cpu_index.train(database)
                ivfpq_end = time.time()
                print("%s Index train time= %0.3f sec" % (index_type, ivfpq_end - ivfpq_start))
                
            elif index_type == 'ivfsq':
                quantizer = faiss.IndexFlatL2(D)
                cpu_index = faiss.IndexIVFScalarQuantizer(quantizer, D, nlist, qtype)
                print("Training IVFSQ Index:  d=%d, qtype=%s, nlist=%d" %(d,qtype,nlist))
                ivfsq_start = time.time()
                cpu_index.train(database)
                ivfsq_end = time.time()
                print("IVFSQ Index train time= %0.3f sec" % (ivfsq_end - ivfsq_start))
                            

            elif index_type in ['kmeans', 'kmeans-pca']:
                ncentroids = dim
                kmeans = faiss.Kmeans(D, ncentroids, verbose=True)
                if use_svd:            
                    database = np.load(root+db_npy_svd)
                            
                print("Learning %s index with d=%d and k=%d" %(index_type, D, ncentroids))
                kmeans_start = time.time()
                kmeans.train(database)
                kmeans_end = time.time()
                print("%s index train time= %0.3f sec" % (index_type, kmeans_end - kmeans_start))           
                cpu_index = kmeans.index
                centroids_path = root+'kmeans/'+model+index_type+'_ncentroids'+str(ncentroids)+"_"+str(D)+'d'"_"+dataset+'.npy'
                print("Saving centroids: ", kmeans.centroids.shape)
                with open(centroids_path, 'wb') as f:
                    np.save(f, kmeans.centroids)

            
            # add database embeddings to the index and save to disk                
            cpu_index.add(database)                  
            faiss.write_index(cpu_index, index_file)

        if use_gpu:
            index = faiss.index_cpu_to_all_gpus(cpu_index)
            print("Moved to GPU")
        else:
            index = cpu_index

        if model != 'mrl/':
            if use_svd:
                query_npy = query_npy_svd
            else:
                query_npy = dataset + '_val_' + config +str(D)+ '-X.npy'
            print("Loading queries: ", query_npy)
            xq = np.load(root+query_npy)
        queryset = np.ascontiguousarray(xq[:,:D], dtype=np.float32)
        faiss.normalize_L2(queryset)
        
        # kmeans indices are used downstream by adanns.ipynb 
        if index_type not in ['kmeans', 'kmeans-pca']:
            print("Searching queries: ", queryset.shape)
            # Loop over search probes/ beam width
            for nprobe in nprobes:
                if index_type != 'pq':
                    index.nprobe=nprobe
                    print("nprobe: ", index.nprobe)
                    
                Dist, Ind = index.search(queryset, k)

                if index_type == 'pq':
                    nn_dir = root+"neighbors/"+model+index_type+"/"+index_type+"_"+str(k)+"shortlist_"+dataset+"_d"+str(D)+".csv"
                else:
                    nn_dir = root+"neighbors/"+model+index_type+"/"+index_type+"_m"+str(dim)+'_nlist'+str(nlist)+'_nprobe'+str(nprobe)+"_"+str(k)+"shortlist_"+dataset+"_d"+str(D)+".csv"
                if not path.isdir(root+"neighbors/"+model+index_type+"/"):
                    makedirs(root+"neighbors/"+model+index_type+"/")
                    
                print(nn_dir.split("/")[-1]+"\n")
                pd.DataFrame(Ind).to_csv(nn_dir, header=None, index=None)
                
                # Optional test for duplicate distances in high M regimes
                #get_duplicate_dist(Ind, Dist)
                
                del Dist, Ind
        del index

Indexing database  (1281167, 8)
Generating index file: 1K_ivfpq_m8_d8_nbits8_nlist1024.index
Loading index file: ../../../inference_array/resnet50/index_files/mrl/ivfpq/1K_ivfpq_m8_d8_nbits8_nlist1024.index
Searching queries:  (50000, 8)
nprobe:  1
ivfpq_m8_nlist1024_nprobe1_2048shortlist_1K_d8.csv

Indexing database  (1281167, 16)
Generating index file: 1K_ivfpq_m8_d16_nbits8_nlist1024.index
Loading index file: ../../../inference_array/resnet50/index_files/mrl/ivfpq/1K_ivfpq_m8_d16_nbits8_nlist1024.index
Searching queries:  (50000, 16)
nprobe:  1
ivfpq_m8_nlist1024_nprobe1_2048shortlist_1K_d16.csv

Indexing database  (1281167, 16)
Generating index file: 1K_ivfpq_m16_d16_nbits8_nlist1024.index
Loading index file: ../../../inference_array/resnet50/index_files/mrl/ivfpq/1K_ivfpq_m16_d16_nbits8_nlist1024.index
Searching queries:  (50000, 16)
nprobe:  1
ivfpq_m16_nlist1024_nprobe1_2048shortlist_1K_d16.csv

Indexing database  (1281167, 32)
Generating index file: 1K_ivfpq_m8_d32_nbits8_nlist

Searching queries:  (50000, 2048)
nprobe:  1
ivfpq_m8_nlist1024_nprobe1_2048shortlist_1K_d2048.csv

Indexing database  (1281167, 2048)
Generating index file: 1K_ivfpq_m16_d2048_nbits8_nlist1024.index
Loading index file: ../../../inference_array/resnet50/index_files/mrl/ivfpq/1K_ivfpq_m16_d2048_nbits8_nlist1024.index
Searching queries:  (50000, 2048)
nprobe:  1
ivfpq_m16_nlist1024_nprobe1_2048shortlist_1K_d2048.csv

Indexing database  (1281167, 2048)
Generating index file: 1K_ivfpq_m32_d2048_nbits8_nlist1024.index
Loading index file: ../../../inference_array/resnet50/index_files/mrl/ivfpq/1K_ivfpq_m32_d2048_nbits8_nlist1024.index
Searching queries:  (50000, 2048)
nprobe:  1
ivfpq_m32_nlist1024_nprobe1_2048shortlist_1K_d2048.csv

Indexing database  (1281167, 2048)
Generating index file: 1K_ivfpq_m64_d2048_nbits8_nlist1024.index
Loading index file: ../../../inference_array/resnet50/index_files/mrl/ivfpq/1K_ivfpq_m64_d2048_nbits8_nlist1024.index
Searching queries:  (50000, 2048)
nprobe:  1